# Notebook de VARIABLES

Este notebook sirve para probar la clasificación de variables de manera automática en base al documento de **Protocolo de Variables**.

## 0a. Ollama

In [2]:
import ollama
import re
from typing import Optional

def consultar_ollama(prompt: str, modelo: str = "gemma3:4b") -> str:
    """
    Función genérica para enviar cualquier prompt a Ollama.
    Devuelve la respuesta del modelo como texto limpio.
    """
    try:
        response = ollama.chat(model=modelo, messages=[
            {
                'role': 'user',
                'content': prompt,
            },
        ])
        return response['message']['content'].strip()
    
    except Exception as e:
        print(f"Error conectando con el modelo {modelo}: {e}")
        return ""

## 0b. Cargar datos

In [3]:
import newspaper
import pandas as pd
from tqdm import tqdm
from pydantic import BaseModel, Field, ValidationError


SUFFIX = "scrape"
data = pd.read_csv(f"../data/2026_02_10_imio_def_todo_envio_heidy.xlsx - 2026_02_09_imio_def_todo_clara_{SUFFIX}.csv")
# data = pd.read_csv("../data/Base de datos Noticias 27102025.xlsx - Noticias_scrape.csv")


data

,no_N_noticia,IdNoticia,Medio_num,Fecha,año,año_agrupado,Caracteres,Titular,nombre_propio_titular,cita_en_titulo,...,no_MES,no_Contenido,no_Pagina_url,no_verificacion,no_textonoticia,antiguo_genero_protagonistas,emocion_ia,IA_razonamiento_titulares,no_vacio_numero_caracteres,contenido_articulo
0,1,1,2,01/01/2024,2024,2,4515,el papa denuncia la violencia contra las mujer...,2,2,...,1.0,"“quien lastima a una mujer profana a dios”, ha...",https://elpais.com/sociedad/2024-01-01/el-papa...,1.0,el papa francisco ha denunciado la violencia c...,3,5.0,El Papa (hombre),NaN,El papa Francisco ha denunciado la violencia c...
1,2,2,2,01/01/2024,2024,2,4880,"inteligencia artificial: agárrense, que vienen...",1,1,...,1.0,los despidos masivos de una industria tan rent...,https://elpais.com/babelia/2024-01-01/intelige...,1.0,"baldur’s gate 3, zelda: tears of the kingdom, ...",4,7.0,NaN,NaN,Artículos estrictamente de opinión que respond...
2,3,3,2,01/01/2024,2024,2,9052,en 2024 votarán miles de millones de personas ...,1,1,...,1.0,la concentración de procesos electorales en un...,https://elpais.com/tecnologia/2024-01-01/en-20...,1.0,"durante el año 2024, el calendario electoral s...",1,7.0,NaN,NaN,Una persona antes de coger una papeleta para e...
3,4,4,2,02/01/2024,2024,2,2523,estupidez artificial,1,1,...,1.0,lo tenebroso no son las nuevas formas de traba...,https://elpais.com/opinion/2024-01-02/estupide...,1.0,lo que da miedo de 2024 no es la inteligencia ...,1,6.0,NaN,NaN,Lo tenebroso no son las nuevas formas de traba...
4,5,5,2,02/01/2024,2024,2,3177,la última navidad,1,1,...,1.0,los lectores escriben sobre la ilusión de los ...,https://elpais.com/opinion/2024-01-02/la-ultim...,1.0,estoy triste. sé positivamente que esta ha sid...,1,3.0,NaN,NaN,Los lectores escriben sobre la ilusión de los ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7110,7111,j995,7,18/01/2017,2018,1,1631,Un ‘pezrobot’ tan realista que puede filmar pe...,1,1,...,NaN,x,https://www.lavanguardia.com/natural/20180322/...,NaN,NaN,4,NaN,No hay.,NaN,La mayoría de los peces robot no se encuentran...
7111,7112,j996,7,18/01/2017,2018,1,2553,YouTube prioriza contenido extremista,5,1,...,NaN,x,https://www.lavanguardia.com/tecnologia/201803...,NaN,NaN,3,NaN,YouTube (empresas).,NaN,NaN
7112,7113,j997,7,05/01/2017,2018,1,4599,Estos son los retos del futuro de la automoción,1,1,...,NaN,x,https://www.lavanguardia.com/motor/20180226/44...,NaN,NaN,4,NaN,No hay.,NaN,El futuro del coche conectado en Volkswagen es...
7113,7114,j998,1,18/01/2017,2021,1,6479,"Lucía Velasco, tecnóloga: ""Hay 15s que recomie...",3,2,...,NaN,x,https://www.elmundo.es/yodona/lifestyle/2021/1...,NaN,NaN,2,NaN,Lucía Velasco (mujer).,NaN,Tú crees que eso de los algoritmos no va conti...


### Artículo

In [4]:
articulo_raw = data[data['IdNoticia'] == "1"].iloc[0]
print(articulo_raw)

# articulo_raw = data[data['IdNoticia'] == 46].iloc[0]
# print(articulo_raw)

no_N_noticia                                                                         1
IdNoticia                                                                            1
Medio_num                                                                            2
Fecha                                                                       01/01/2024
año                                                                               2024
año_agrupado                                                                         2
Caracteres                                                                        4515
Titular                              el papa denuncia la violencia contra las mujer...
nombre_propio_titular                                                                2
cita_en_titulo                                                                       2
cla_genero_prota                                                                     3
nombre_periodista                          

#### Texto del artículo

In [5]:
# articulo_text = articulo['no_Contenido']
articulo_text = articulo_raw['contenido_articulo']

articulo_text

'El papa Francisco ha denunciado la violencia contra las mujeres en la misa del día de Año Nuevo, y ha defendido que la Iglesia debe darle más espacio a ellas y “redescubrir su rostro femenino”. “Quien lastima a una mujer profana a Dios, nacido de mujer”, condenó Bergoglio. Y recordó: “Toda sociedad necesita acoger el don de la mujer, de cada mujer: respetarla, cuidarla, valorarla”.\n\nEl día 1 de enero el cristianismo dedica el día a la Virgen, por eso el Pontífice abrió el año con una misa en la basílica en la que ensalzó el papel de María y de la mujer en la Iglesia. Francisco hizo referencia en su discurso a textos papales, como la encíclica Lumen gentium (1964) que Pablo VI escribió en el revolucionario Concilio Vaticano II, y señaló que “la Iglesia necesita de María para redescubrir su propio rostro femenino, para asemejarse más a ella que, como mujer, Virgen y Madre, representa su modelo y su figura perfecta; para dar espacio a las mujeres y para ser generativa a través de una p

#### URL del artículo

In [6]:
articulo_url = articulo_raw['no_Pagina_url']
# articulo_url = articulo_raw['Pagina']

articulo_url

'https://elpais.com/sociedad/2024-01-01/el-papa-denuncia-la-violencia-contras-las-mujeres-y-reclama-mas-espacio-para-ellas-en-la-iglesia.html'

#### Objeto Article de newspaper

In [7]:
from newspaper import Article

# Crear el objeto Article
# Puedes añadir language='es' si ayuda al parser, aunque suele detectarlo solo.
articulo = Article(articulo_url, language='es')

try:
    # 2. Descargar el HTML
    articulo.download()
    
    # 3. Parsear (analizar) el contenido
    articulo.parse()

except Exception as e:
    print(f"Error al procesar la URL: {e}")

---

# 02/2026 - Variables Generales

---

## 1. ID noticia

In [8]:
def obtener_id_noticia(articulo_raw):
    """
    Extrae y devuelve el IdNoticia de un registro dado.
    """
    articulo_id = articulo_raw['IdNoticia']
    return articulo_id

In [9]:
# --- Uso ---
articulo_id = obtener_id_noticia(articulo_raw)
print(f"{articulo_id=}")

articulo_id='1'


## 2. Medio

In [10]:
from urllib.parse import urlparse

# Función para clasificar el medio según la URL
def clasificar_var_medio(articulo):
    # Extraer el dominio de la URL
    url = articulo.url
    domain = urlparse(url).netloc.lower()
    
    # Diccionario de dominios y categorías
    media_map = {
        "elmundo.es": 1,
        "elpais.com": 2,
        "eldiario.es": 3,
        "20minutos.es": 4,
        "articulo14.com": 5,
        "infolibre.es": 6,
        "lavanguardia.com": 7
    }
    
    # Buscar el dominio en el mapa
    for key, value in media_map.items():
        if key in domain:
            return key, value
    
    # Si no coincide con ninguno, retornar None o 0
    return 0

In [11]:
# --- Uso ---
Medio = clasificar_var_medio(articulo)[0]
Medio_num = clasificar_var_medio(articulo)[1]

print(f"Medio: {Medio}")
print(f"Categoría: {Medio_num}")

Medio: elpais.com
Categoría: 2


## 3. Fecha

In [12]:
from typing import Optional
from newspaper import Article

def clasificar_var_fecha(articulo: Article) -> Optional[str]:
    """
    Recibe un objeto Article.
    Devuelve la fecha como texto en formato 'dd/mm/aa' o None.
    """
    # Verificamos si existe la fecha
    if articulo.publish_date:
        # %d = día (01-31)
        # %m = mes (01-12)
        # %y = año (dos últimos dígitos, ej: 26)
        return articulo.publish_date.strftime("%d/%m/%y")
    
    return None

In [13]:
# --- Uso ---
Fecha = clasificar_var_fecha(articulo)
print(f"Fecha: {Fecha}")

Fecha: 01/01/24


## 3a. Mes

In [14]:
def clasificar_var_mes(articulo: Article) -> Optional[int]:
    """
    Recibe un objeto Article procesado.
    Devuelve el mes (MM) como entero o None si no tiene fecha.
    """
    if articulo.publish_date:
        return articulo.publish_date.month
    
    return None

In [15]:
# --- Uso ---
mes = clasificar_var_mes(articulo)
print(f"mes: {mes}")

mes: 1


## 4. Año

In [16]:
def clasificar_var_año(articulo: Article) -> Optional[int]:
    """
    Recibe un objeto Article procesado.
    Devuelve el año (YYYY) como entero o None si no tiene fecha.
    """
    if articulo.publish_date:
        return articulo.publish_date.year
    
    return None

In [17]:
# --- Uso ---
año = clasificar_var_año(articulo)
print(f"año: {año}")

año: 2024


## 5. Número de caracteres

In [18]:
def clasificar_var_caracteres(articulo: Article) -> int:
    """
    Devuelve la cantidad de caracteres del cuerpo del artículo.
    Si no hay texto, devuelve 0.
    """
    if articulo.text:
        # Retorna la longitud del texto
        return len(articulo.text)
    
    return 0

In [19]:
# --- Uso ---
Caracteres = clasificar_var_caracteres(articulo)
print(f"Caracteres: {Caracteres}")

Caracteres: 4697


## 6. Titular. Copia el titular

In [20]:
def clasificar_var_titular(articulo: Article) -> Optional[str]:
    """
    Recibe un objeto Article procesado.
    Devuelve el titular en minúsculas o None si no existe.
    """
    if articulo.title:
        # .strip() quita espacios extra y .lower() pasa a minúsculas
        return articulo.title.strip().lower()
    
    return None

In [21]:
# --- Uso ---
Titular = clasificar_var_titular(articulo)
print(f"Titular: {Titular}")

Titular: el papa denuncia la violencia contra las mujeres y reclama más “espacio” para ellas en la iglesia


## 7a. Nombre Propio Titular

In [40]:
from typing import List
import json

class NombresDetectados(BaseModel):
    # Lista de cadenas con los nombres extraídos
    nombres: List[str] = Field(default_factory=list, description="Lista de nombres propios detectados")
    # Lista de enteros con los códigos correspondientes
    valores: List[int] = Field(default_factory=list, description="Lista de valores clasificados según la tabla")

def clasificar_var_nombre_propio_titular_list(titulo: str) -> NombresDetectados:
    """
    Extrae nombres propios del titular y los clasifica según su género o tipo.
    Devuelve dos listas sincronizadas: nombres y valores.
    """
    
    # Si el título es muy corto, devolvemos listas vacías
    if not titulo or len(titulo) < 3:
        return NombresDetectados(nombres=[], valores=[])

    prompt = f"""
    Analiza el siguiente TITULAR y extrae los nombres propios, entidades, lugares o tecnologías.
    Asigna a cada uno su código numérico correspondiente.

    TITULAR: "{titulo}"

    TABLA DE CÓDIGOS:
    1  = Hombre (Nombre individual)
    2  = Mujer (Nombre individual)
    3  = Grupo Mixto (ej: "Los Reyes", "La pareja", "Padres")
    32 = Grupo Mixto (Mayoría hombres)
    33 = Grupo Mixto (Mayoría mujeres)
    4  = Institución, Organización, Empresa, Partido Político (ej: "Google", "PSOE", "La ONU")
    41 = Lugares: Países, Regiones, Ciudades (ej: "España", "Madrid", "Europa")
    42 = Tecnología: Apps, Modelos de IA, Robots, Software (ej: "ChatGPT", "Gemini", "Sora", "TikTok")

    INSTRUCCIONES:
    - Ignora sustantivos comunes que no sean entidades (ej: no extraigas "la policía" si es genérico, pero sí "Mossos d'Esquadra").
    - Distingue bien entre la EMPRESA (OpenAI -> 4) y el PRODUCTO (ChatGPT -> 42).
    - Si no hay nombres propios, devuelve listas vacías.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON con esta estructura exacta:
    {{
        "nombres": ["Nombre1", "Nombre2"],
        "valores": [Codigo1, Codigo2]
    }}
    """

    # --- LLAMADA AL MODELO ---
    respuesta_texto = consultar_ollama(prompt)

    # --- PARSEO Y VALIDACIÓN ---
    try:
        # Limpieza básica para encontrar el JSON
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1:
            return NombresDetectados(nombres=[], valores=[])
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)
        
        # Validamos con Pydantic
        resultado = NombresDetectados(**data)
        
        # Validación de seguridad: Las listas deben tener el mismo tamaño
        if len(resultado.nombres) != len(resultado.valores):
            # Si hay desajuste, cortamos a la longitud del más corto
            min_len = min(len(resultado.nombres), len(resultado.valores))
            resultado.nombres = resultado.nombres[:min_len]
            resultado.valores = resultado.valores[:min_len]
            
        return resultado

    except (json.JSONDecodeError, ValidationError):
        return NombresDetectados(nombres=[], valores=[])


In [ ]:
nombre_propio_titular_list = clasificar_var_nombre_propio_titular_list(articulo.title)
print(f"Nombre Propio Titular Lista Nombres: {nombre_propio_titular_list.nombres}")
print(f"Nombre Propio Titular Lista Valores: {nombre_propio_titular_list.valores}")

Nombre Propio Titular Lista Nombres: ['Papa']
Nombre Propio Titular Lista Valores: [1]


## 7b. Género Nombre Propio Titular

In [ ]:
def clasificar_var_nombre_propio_titular(valores: list[int]) -> int:
    """
    Calcula el valor único de protagonismo basado en la lista de entidades detectadas.
    Prioridad absoluta a las personas sobre entidades/lugares.
    """
    
    if not valores:
        return 0

    # 1. CONTADORES DE PERSONAS
    cnt_hombres = valores.count(1)
    cnt_mujeres = valores.count(2)
    
    # Contamos también si el LLM ya detectó grupos mixtos explícitos
    cnt_mixtos_neutro = valores.count(3)
    cnt_mixtos_hombres = valores.count(32)
    cnt_mixtos_mujeres = valores.count(33)

    # Suma total de indicadores humanos
    total_humanos = cnt_hombres + cnt_mujeres + cnt_mixtos_neutro + cnt_mixtos_hombres + cnt_mixtos_mujeres

    # --- FASE 1: FILTRO HUMANO (Prioridad Absoluta) ---
    if total_humanos > 0:
        
        # CASO A: Solo Hombres (Sin mujeres ni grupos mixtos)
        if cnt_hombres > 0 and cnt_mujeres == 0 and cnt_mixtos_neutro == 0 and cnt_mixtos_hombres == 0 and cnt_mixtos_mujeres == 0:
            return 1
            
        # CASO B: Solo Mujeres (Sin hombres ni grupos mixtos)
        if cnt_mujeres > 0 and cnt_hombres == 0 and cnt_mixtos_neutro == 0 and cnt_mixtos_hombres == 0 and cnt_mixtos_mujeres == 0:
            return 2

        # CASO C: Mixto (Hay presencia de ambos o indicadores de grupo)
        # Aquí decidimos si es 3, 32 o 33 contando individuos
        
        if cnt_hombres > cnt_mujeres:
            return 32  # Mixto mayoritariamente masculino
        elif cnt_mujeres > cnt_hombres:
            return 33  # Mixto mayoritariamente femenino
        else:
            # Empate técnico (ej: 1 hombre y 1 mujer) o solo había un [3] genérico
            # Si el empate viene de counts explícitos (1 vs 1), es 3.
            # Si viene de grupos (ej: un [32] detectado), respetamos ese matiz.
            if cnt_mixtos_hombres > cnt_mixtos_mujeres:
                return 32
            elif cnt_mixtos_mujeres > cnt_mixtos_hombres:
                return 33
            else:
                return 3 # Empate total o mixto neutro

    # --- FASE 2: NO HUMANOS (Solo si total_humanos == 0) ---
    # Establecemos jerarquía de interés: IA > Entidad > Lugar
    
    if 42 in valores:
        return 42 # Prioridad a Tecnología/IA
    
    if 4 in valores:
        return 4  # Empresas / Instituciones
        
    if 41 in valores:
        return 41 # Lugares (es lo menos informativo)

    # Si todo falla (ej: llegó un código desconocido)
    return 0

In [45]:
nombre_propio_titular = clasificar_var_nombre_propio_titular(nombre_propio_titular_list.valores)
print("nombre_propio_titular", nombre_propio_titular)

nombre_propio_titular 1


In [22]:
import requests
from flair.models import SequenceTagger
from flair.data import Sentence

# 1. Cargar el modelo (se hace una sola vez)
tagger = SequenceTagger.load("flair/ner-spanish-large")

# Caché para no repetir llamadas a la API y ahorrar créditos
cache_generos = {}

def get_gender_cached(name):
    """Consulta Genderize y guarda el resultado en memoria."""
    name = name.lower().capitalize()
    if name in cache_generos:
        return cache_generos[name]
    try:
        # Usamos timeout para que no se quede colgado si falla la red
        response = requests.get(f"https://api.genderize.io?name={name}", timeout=5)
        if response.status_code == 200:
            gen = response.json().get("gender")
            cache_generos[name] = gen
            return gen
    except:
        return None
    return None

def clasificar_var_nombre_propio_titular(titular: str):
    """
    Detecta nombres y devuelve (categoría, lista_nombres).
    Categorías: 1=No hay, 2=Hombre, 3=Mujer, 4=Ambos, 5=Neutro (entidades)
    """
    if not titular:
        return 1, []

    sentence = Sentence(titular)
    tagger.predict(sentence)

    nombres_personas = []
    otras_entidades = []

    for entity in sentence.get_spans('ner'):
        if entity.tag == "PER":
            nombres_personas.append(entity.text)
        else:
            otras_entidades.append(entity.text)

    # Lógica de categorías
    if not nombres_personas and otras_entidades:
        return 5, []
    
    if not nombres_personas:
        return 1, []

    # Determinamos género para la categoría
    hombres, mujeres = 0, 0
    for np in nombres_personas:
        primer_nombre = np.split()[0]
        gen = get_gender_cached(primer_nombre)
        if gen == "male": hombres += 1
        elif gen == "female": mujeres += 1

    if hombres == 0 and mujeres == 0: cat = 1
    elif hombres > mujeres: cat = 2
    elif mujeres > hombres: cat = 3
    else: cat = 4 # Empate o presencia de ambos

    return cat, nombres_personas

def clasificar_var_genero_nombre_propio_titular(lista_nombres):
    """
    Recibe la lista de nombres y devuelve sus géneros.
    Categorías: 1=No hay, 2=Hombre, 3=Mujer, 4=Ambos, 5=Neutro (entidades)
    """
    generos = []
    for nombre_completo in lista_nombres:
        primer_nombre = nombre_completo.split()[0]
        genero = get_gender_cached(primer_nombre)
        # Traducimos a algo más legible
        if genero == "male": generos.append("2")
        elif genero == "female": generos.append("3")
        else: generos.append("desconocido")
    return generos


/home/jggomez/Desktop/IRIS/iris-uc3m/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-02-14 20:40:16.241372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771098016.257255 3425484 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771098016.262184 3425484 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-14 20:40:16.278434: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in perfor

2026-02-14 20:40:30,215 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


In [23]:
# --- Ejemplo de uso ---
# Titular = "Isabel Díaz Ayuso se reúne con Pedro Sánchez en Madrid"

# # 1. Obtenemos categoría y nombres
# nombre_propio_titular, nombre_propio_titular_lista = clasificar_var_nombre_propio_titular(Titular)

# # 2. Obtenemos los géneros basándonos en esa lista
# genero_nombre_propio_titular = clasificar_var_genero_nombre_propio_titular(nombre_propio_titular_lista)

# print(f"nombre_propio_titular: {nombre_propio_titular}")
# print(f"nombres_encontrados: {nombre_propio_titular_lista}")
# print(f"genero_nombre_propio_titular: {genero_nombre_propio_titular}")


# 1. Obtenemos categoría y nombres
nombre_propio_titular, nombre_propio_titular_lista = clasificar_var_nombre_propio_titular(Titular)

# 2. Obtenemos los géneros basándonos en esa lista
genero_nombre_propio_titular = clasificar_var_genero_nombre_propio_titular(nombre_propio_titular_lista)

print(f"nombre_propio_titular: {nombre_propio_titular}")
print(f"nombres_encontrados: {nombre_propio_titular_lista}")
print(f"genero_nombre_propio_titular: {genero_nombre_propio_titular}")

nombre_propio_titular: 1
nombres_encontrados: []
genero_nombre_propio_titular: []


## 8. Cita en el titular

In [24]:
import re
from flair.data import Sentence
from flair.models import SequenceTagger
from transformers import pipeline
from IPython.display import display, HTML

# Cargar modelo de NER en español (Flair)
tagger = SequenceTagger.load("flair/ner-spanish-large")

# Modelo opcional Zero-Shot para clasificar citas
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Función para extraer fuentes con NER
def extract_sources(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
   
    sources = set()
    for entity in sentence.get_spans("ner"):
        if entity.tag in ["PER", "ORG"]:  # Solo personas y organizaciones
            sources.add(entity.text)
   
    return list(sources)

# Función para extraer citas directas (entre comillas)
def extract_explicit_quotes(text):
    pattern = r'(["“][^"“”]+["”])'  # Busca texto entre comillas
    quotes = re.findall(pattern, text)
    return [{"source": "Desconocido", "quote": quote} for quote in quotes]

# Función para extraer citas indirectas junto con su fuente
def extract_indirect_quotes(text):
    pattern = r'([A-ZÁÉÍÓÚ][a-záéíóú]+(?:\s[A-ZÁÉÍÓÚ][a-záéíóú]+)?)?\s*(dijo|afirmó|expresó|mencionó|declaró|aseguró|comentó|indicó)\s*([^\.\n]+)'
    matches = re.findall(pattern, text, re.IGNORECASE)
   
    extracted_quotes = []
    for match in matches:
        source = match[0].strip() if match[0] else "Desconocido"
        verb = match[1]
        quote = match[2].strip()
       
        extracted_quotes.append({"source": source, "verb": verb, "quote": quote})
   
    return extracted_quotes

# Función para validar con zero-shot classification
def validate_with_zero_shot(text):
    candidate_labels = ["cita", "sin cita"]
    result = zero_shot_classifier(text, candidate_labels)
    return result["labels"][0] == "cita"

# Función combinada para detectar citas junto con la fuente
def detect_and_extract_quotes(text, use_zero_shot=False):
    explicit_quotes = extract_explicit_quotes(text)
    indirect_quotes = extract_indirect_quotes(text)
   
    sources = extract_sources(text)

    has_quote = bool(explicit_quotes or indirect_quotes)
    if not has_quote and use_zero_shot:
        has_quote = validate_with_zero_shot(text)
   
    return {
        "has_quote": int(has_quote),
        "explicit_quotes": explicit_quotes,
        "indirect_quotes": indirect_quotes,
        "sources": sources
    }

# Función para resaltar citas y fuentes en HTML
def highlight_quotes_html(text):
    text = re.sub(r'(["“][^"“”]+["”])', r'<span class="explicit-quote">\1</span>', text)
    text = re.sub(r'([A-ZÁÉÍÓÚ][a-záéíóú]+(?:\s[A-ZÁÉÍÓÚ][a-záéíóú]+)?)?\s*(dijo|afirmó|expresó|mencionó|declaró|aseguró|comentó|indicó)\s*([^\.\n]+)',
                  r'<span class="source">\1</span> <span class="verb">\2</span> <span class="indirect-quote">\3</span>', text, flags=re.IGNORECASE)
    return text

2026-02-14 20:40:40,618 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


/home/jggomez/Desktop/IRIS/iris-uc3m/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
# Detectar y extraer citas
quote_info = detect_and_extract_quotes(Titular)
print(f"(Pred) Cita en el titular: {quote_info}")
print(f"(Real) Cita en el titular: {articulo_raw['cita_en_titulo']}")

# Generar HTML con citas y fuentes resaltadas
html_content = f"""
<style>
    .explicit-quote {{ color: red; font-weight: bold; }}
    .indirect-quote {{ color: blue; font-style: italic; }}
    .source {{ color: green; font-weight: bold; }}
    .verb {{ color: purple; }}
</style>
<p>{highlight_quotes_html(Titular)}</p>
"""

# Mostrar HTML en Jupyter Notebook
display(HTML(html_content))

(Pred) Cita en el titular: {'has_quote': 1, 'explicit_quotes': [{'source': 'Desconocido', 'quote': '“espacio”'}], 'indirect_quotes': [], 'sources': []}
(Real) Cita en el titular: 2


## 9. Sexo personas que aparecen en la información (y personas que aparecen en la información)

### NER (tag de persona)

In [26]:
from flair.models import SequenceTagger
from flair.data import Sentence
import requests

# Configurar el modelo de NER (Reconocimiento de Entidades Nombradas)
tagger = SequenceTagger.load("flair/ner-spanish-large")

# Procesar el texto con Flair
sentence = Sentence(articulo_text)
tagger.predict(sentence)

# Extraer entidades PERSON
names = [entity.text for entity in sentence.get_spans('ner') if entity.tag == "PER"]
print("Nombres detectados:", names)

# Contar géneros
genders = {"male": 0, "female": 0}

# Procesar los nombres detectados
for full_name in names:
    first_name = get_first_name(full_name)  # Extraer el primer nombre
    gender = get_gender_with_genderize(first_name)  # Determinar género
    print(f"Nombre completo: {full_name}, Género: {gender}")
    if gender == "male":
        genders["male"] += 1
    elif gender == "female":
        genders["female"] += 1

# Clasificar el género predominante
if genders["male"] > 0 and genders["female"] == 0:
    category = 0  # Masculino
elif genders["female"] > 0 and genders["male"] == 0:
    category = 1  # Femenino
elif genders["male"] > 0 and genders["female"] > 0:
    category = 2  # Mixto
else:
    category = 3  # Neutro (no se detectaron personas)

print("\n")
print(f"(Pred) Género predominante en la información: {category}")
print(f"(Real) Género predominante en la información: {articulo_raw['cla_genero_prota']}")


2026-02-14 20:40:52,183 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>
Nombres detectados: ['Francisco', 'Bergoglio', 'Pontífice', 'María', 'Francisco', 'Pablo VI', 'María', 'Pontífice', 'María', 'Francisco', 'papa', 'Francisco', 'REMO CASILLI', 'Francisco', 'Alessandra Smerilli', 'Francisco', 'Papa', 'Pontífice', 'Francisco']


NameError: name 'get_first_name' is not defined

### Zero-shot

In [ ]:
from transformers import pipeline

# Configurar modelo Zero-Shot
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Etiquetas candidatas
candidate_labels = ["solo hombres", "solo mujeres", "mixto", "neutro"]

# Clasificación Zero-Shot
result = zero_shot_classifier(articulo_text, candidate_labels)
print("Clasificación Zero-Shot:", result["labels"][0])  # Categoría más probable


Clasificación Zero-Shot: mixto


## 10. Nombre Periodista

In [ ]:
def clasificar_var_nombre_periodista(articulo: articulo) -> str:
    """
    Extrae autores y limpia textos basura como 'Ver Biografía', 'Redacción', etc.
    """
    
    # Lista de palabras/frases que NO queremos en el nombre
    palabras_basura = [
        "ver biografía", "biografía", "ver perfil", "perfil", 
        "ver más", "see profile", "read more", "twitter", 
        "email", "follow", "redacción", "agencia"
    ]

    autores_detectados = []

    # 1. Intentar obtener autores desde el parser
    if articulo.authors:
        for autor in articulo.authors:
            autor_limpio = autor.strip()
            
            # Verificamos si el texto (en minúsculas) contiene alguna palabra basura
            if any(basura in autor_limpio.lower() for basura in palabras_basura):
                # Si es basura pura (ej: "Ver Biografía"), lo ignoramos
                # Pero si es "Lorena Pacho, Ver Biografía", intentamos limpiarlo
                
                # Caso específico que te pasó: eliminar "Ver Biografía" del string
                for basura in palabras_basura:
                    # Reemplazamos la basura por vacío, ignorando mayúsculas/minúsculas es complejo,
                    # así que hacemos un replace simple de las variantes comunes:
                    autor_limpio = autor_limpio.replace("Ver Biografía", "")
                    autor_limpio = autor_limpio.replace("Ver biografía", "")
                
                autor_limpio = autor_limpio.strip(" ,|") # Limpiamos comas o barras sobrantes

            # Si después de limpiar queda algo y no es demasiado corto, lo guardamos
            if len(autor_limpio) > 2:
                autores_detectados.append(autor_limpio)

    # 2. Si encontramos autores limpios, los devolvemos
    if autores_detectados:
        # Eliminamos duplicados usando set() y mantenemos orden
        return ", ".join(list(dict.fromkeys(autores_detectados)))

    # 3. Fallback: Metadatos (si la lista authors falló o se limpió todo)
    meta = articulo.meta_data
    claves_meta = ['author', 'og:author', 'dc.creator', 'byl']
    
    for clave in claves_meta:
        valor = meta.get(clave)
        if valor:
            # A veces los metadatos también traen basura, podrías aplicar limpieza aquí también
            return str(valor).strip()

    # 4. Defecto
    return "Redacción / Otros"

In [ ]:
# --- Uso ---
nombre_periodista = clasificar_var_nombre_periodista(articulo)
print(f"nombre_periodista: {nombre_periodista}")

nombre_periodista: Lorena Pacho


## 11. Género Periodista (Autoría)

In [ ]:
from pydantic import BaseModel, Field, ValidationError
from typing import Optional
import re

# Modelo de validación estricta
class GeneroPeriodistaValidado(BaseModel):
    # Field(...) hace el campo obligatorio
    # ge=0: Greater or equal to 0
    # le=5: Less or equal to 5
    codigo: int = Field(..., ge=0, le=7, description="Código de clasificación de autoría (0-7)")


def clasificar_var_genero_periodista(nombre_periodista: str, nombre_medio:str) -> int:
    """
    Clasifica la autoría considerando el contexto del medio.
    Recibe:
      - nombre_periodista: El texto de la firma (ej: "Redacción", "Juan Pérez")
      - nombre_medio: El nombre del periódico donde se publica (ej: "El País")
    """
    
    # Validación inicial rápida
    if not nombre_periodista or len(nombre_periodista) < 2:
        return 0

    # Prompt enriquecido con el contexto del medio y definiciones
    prompt = f"""
    Contexto:
    Noticia publicada en el medio: "{nombre_medio}".
    Autor/Firma a analizar: "{nombre_periodista}".
    
    Tu misión es clasificar la autoría (0-7) siguiendo estrictamente estas definiciones:

    0 = Ns/Nc: Desconocido, ambiguo o iniciales.
    1 = Hombre: Nombre de persona masculino.
    2 = Mujer: Nombre de persona femenino.
    3 = Mixto: Varios autores de distinto género.
    4 = Otros medios: El autor es otro medio de comunicación (ej: "The New York Times", "Revista Hola").
    5 = Agencia: Agencias de noticias puras (EFE, Europa Press, Reuters, AFP).
    
    6 = Redacción (Periodística): 
        - Firma genérica del propio medio "{nombre_medio}" (ej: "Redacción", "El País", "Editorial").
        - IMPORTANTE: Si el autor es una empresa comercial que NO es un medio de noticias (como Ford, Apple), NO ES 6.
    
    7 = Corporativo (Comercial / Institucional): 
        - Firmado por una empresa comercial, marca de tecnología, coches, banco, etc. (ej: Ford, Meta, Google, BBVA, Zara).
        - Firmado por instituciones gubernamentales o ONGs (ej: Gobierno de España, Greenpeace).
        - Notas de prensa firmadas por la marca.

    INSTRUCCIONES DE PRIORIDAD:
    1. Si "{nombre_periodista}" es una marca conocida (coches, tech, ropa) -> ELIGE 7.
    2. Si "{nombre_periodista}" es igual a "{nombre_medio}" -> ELIGE 6.
    3. Si "{nombre_periodista}" es una Agencia conocida -> ELIGE 5.

    Responde ÚNICAMENTE con el número dígito (0-7).
    """
    # 1. Llamada al modelo (tu función externa)
    respuesta_texto = consultar_ollama(prompt)

    # 2. Extracción del número (Pre-procesamiento)
    # Buscamos el primer dígito que aparezca en el texto
    match = re.search(r'\d+', respuesta_texto)
    
    numero_detectado = int(match.group()) if match else 0

    # 3. Validación con Pydantic
    try:
        # Instanciamos el modelo con el dato detectado
        # Si numero_detectado es 9 (alucinación), Pydantic dará error aquí
        resultado = GeneroPeriodistaValidado(codigo=numero_detectado)
        
        # Si llegamos aquí, es un int válido entre 0 y 5
        return resultado.codigo

    except ValidationError as e:
        print(f"Error de validación Pydantic (Dato inválido: {numero_detectado}): {e}")
        return 0 # Default seguro si la IA alucina un número fuera de rango

In [ ]:
# -- Uso -- 
genero_periodista = clasificar_var_genero_periodista(nombre_periodista=nombre_periodista, nombre_medio=Medio)
print(f"{genero_periodista=}")

genero_periodista=2


## 12. Tema

In [ ]:
import json


class TemaConExplicacion(BaseModel):
    # Validamos que sea un entero entre 0 y 17
    codigo: int = Field(..., ge=0, le=17, description="Código numérico del tema")
    # Añadimos el campo de explicación
    explicacion: str = Field(..., description="Breve justificación de por qué se eligió este tema")

def clasificar_var_tema(titulo: str, texto_cuerpo: str) -> TemaConExplicacion:
    """
    Clasifica el tema y da una explicación.
    Devuelve un objeto Pydantic con .codigo (int) y .explicacion (str).
    """
    
    # 1. Validación rápida
    full_text = f"{titulo} {texto_cuerpo}"
    if not full_text or len(full_text) < 10:
        # Devolvemos objeto vacío/error
        return TemaConExplicacion(codigo=0, explicacion="Texto insuficiente para clasificar.")

    # Recorte para optimizar velocidad
    texto_recortado = texto_cuerpo[:1500]
    
    # 2. Prompt diseñado para JSON
    prompt = f"""
    Analiza la noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea es clasificarla en UNA categoría (1-17) y explicar por qué.
    
    Categorías:
    1 = Científica / Investigación
    2 = Comunicación
    3 = De farándula o espectáculo
    4 = Deportiva
    5 = Economía (Mercados, inflación, consumo)
    6 = Educación/cultura
    7 = Empleo/Trabajo
    8 = Empresa (Corporativo, negocios)
    9 = Judicial
    10 = Medioambiente
    11 = Policial
    12 = Política
    13 = Salud
    14 = Social
    15 = Tecnología
    16 = Transporte
    17 = Otros

    FORMATO DE RESPUESTA OBLIGATORIO (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido con este formato:
    {{
        "codigo": (número entero del 1 al 17),
        "explicacion": "(frase breve justificando tu elección)"
    }}
    """

    # 3. Llamada al modelo
    respuesta_texto = consultar_ollama(prompt)

    # 4. Limpieza y Parsing de JSON
    # A veces los modelos envuelven el JSON en markdown ```json ... ```
    # Buscamos donde empieza '{' y termina '}'
    try:
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            raise ValueError("No se encontró JSON en la respuesta")
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str) # Convertimos texto a diccionario Python

        # 5. Validación Pydantic
        resultado = TemaConExplicacion(**data)
        return resultado

    except (json.JSONDecodeError, ValueError, ValidationError) as e:
        print(f"Error parseando respuesta del modelo: {e}")
        # Retorno de seguridad en caso de fallo
        return TemaConExplicacion(codigo=0, explicacion=f"Error técnico: {str(e)}")

In [ ]:
# -- Uso --
tema = clasificar_var_tema(articulo.title, articulo.text)
print(f"ID Tema: {tema.codigo}")
print(f"Explicación: {tema.explicacion}")

ID Tema: 14
Explicación: La noticia trata sobre la denuncia de violencias contra las mujeres, el papel de la mujer en la Iglesia, y la búsqueda de paz y humanidad, lo que lo clasifica claramente como un tema social.


## 12b. Sección

In [ ]:
def clasificar_var_seccion(articulo: Article) -> str:
    """
    Extrae la sección del periódico basándose en metadatos y la URL.
    No usa IA (es más rápido y exacto para este dato estructural).
    """
    
    # --- 1. BUSCAR EN METADATOS (La fuente más fiable) ---
    meta = articulo.meta_data
    
    # Lista de claves comunes donde los medios guardan la sección
    claves_seccion = [
        'section',           # Estándar simple
        'article:section',   # Protocolo Open Graph (Facebook/LinkedIn)
        'og:section',        # Variación Open Graph
        'category',          # WordPress y CMS comunes
        'dc.subject',        # Dublin Core standard
        'ut.section'         # Algunos medios custom
    ]

    for clave in claves_seccion:
        valor = meta.get(clave)
        # A veces el valor es una lista, tomamos el primero
        if isinstance(valor, list):
            valor = valor[0]
        
        if valor and isinstance(valor, str) and len(valor) > 2:
            return valor.strip().title() # Ej: "DEPORTES " -> "Deportes"

    # --- 2. BUSCAR EN LA URL (Si no hay metadatos) ---
    # Ejemplo: https://www.elmundo.es/economia/2024/02/10/noticia.html
    # Queremos extraer "economia"
    
    path = urlparse(articulo.url).path
    segmentos = path.split('/')
    
    # Filtramos segmentos vacíos
    segmentos = [s for s in segmentos if s]

    for segmento in segmentos:
        # Ignoramos segmentos que son años (4 dígitos) o muy cortos (idiomas 'es', 'en')
        if re.match(r'^\d{4}$', segmento): # Es un año (2024)
            continue
        if re.match(r'^\d{1,2}$', segmento): # Es un día o mes (10, 02)
            continue
        if len(segmento) <= 2: # Es un código de idioma (es, en, cat)
            continue
        if segmento in ['noticia', 'articulo', 'story', 'news']: # Palabras genéricas
            continue
            
        # Si pasa los filtros, asumimos que es la sección
        # Reemplazamos guiones por espacios (ej: "ciencia-y-salud" -> "Ciencia Y Salud")
        return segmento.replace('-', ' ').title()

    # --- 3. DEFECTO ---
    return "General"

In [ ]:
# -- Uso --
seccion = clasificar_var_seccion(articulo)
print(f"Sección detectada: {seccion}")

Sección detectada: Sociedad


----

# 02/2026 - Variables específicas IRIS

---

## 13. IA tema central

In [ ]:
class IaTemaCentralConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No es tema central, 2=Sí es tema central")
    # Campo nuevo
    explicacion: str = Field(..., description="Justificación de la jerarquía de la información")

def clasificar_var_ia_tema_central(titulo: str, texto_cuerpo: str) -> IaTemaCentralConExplicacion:
    """
    Determina si la Inteligencia Artificial es el TEMA CENTRAL de la noticia.
    Devuelve objeto con .codigo y .explicacion.
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()

    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    palabras_clave_ia = [
        "inteligencia artificial", "artificial intelligence", "ia ", "ai ", 
        "chatgpt", "gpt", "llm", "machine learning", "aprendizaje automático",
        "red neuronal", "deep learning", "midjourney", "dall-e", "bard", "gemini",
        "copilot", "algoritmo generativo", "sam altman", "openai", "nvidia"
    ]
    
    # Si ninguna palabra clave está presente, asumimos directamente que NO (1)
    if not any(palabra in texto_completo for palabra in palabras_clave_ia):
        return IaTemaCentralConExplicacion(
            codigo=1,
            explicacion="El texto no contiene términos relacionados con la Inteligencia Artificial."
        )

    # --- 2. PROMPT CON SOLICITUD DE JSON ---
    # Recortamos texto para centrar la atención en el inicio (donde suele estar el tema central)
    texto_recortado = texto_cuerpo[:2500]
    
    prompt = f"""
    Analiza la jerarquía de la información en esta noticia:
    
    TÍTULO: "{titulo}"
    TEXTO: "{texto_recortado}..."

    Objetivo: Determinar si la Inteligencia Artificial (IA) es el TEMA PRINCIPAL y PROTAGONISTA.
    
    Criterios de clasificación:

    1 = No (Mención secundaria / Otro tema):
        - La IA se menciona al final o de pasada.
        - Es un discurso (Papa, Políticos) sobre varios temas y la IA es solo uno más.
        - La IA es una herramienta secundaria (ej: "Policía usa IA para un robo", el tema es el robo).
        - El Título NO menciona tecnología o IA.

    2 = Sí (Tema Central):
        - La noticia gira completamente en torno a la IA (avances, regulación, peligros, inversiones).
        - La IA es el sujeto principal del Título.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Breve justificación analizando si la IA es protagonista o secundaria)"
    }}
    """

    # --- 3. LLAMADA AL MODELO ---
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. EXTRACCIÓN Y VALIDACIÓN ---
    try:
        # Buscamos el bloque JSON
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            return IaTemaCentralConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato JSON válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        # Validación Pydantic
        return IaTemaCentralConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        # Fallback seguro
        return IaTemaCentralConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
ia_tema_central = clasificar_var_ia_tema_central(articulo.title, articulo.text)

print(f"Código: {ia_tema_central.codigo}")
print(f"Razón: {ia_tema_central.explicacion}")

Código: 1
Razón: La noticia se centra en el discurso del Papa Francisco sobre la violencia contra las mujeres, el papel de la mujer en la Iglesia y la necesidad de 'espacio' para ellas. La IA se menciona brevemente al final, en relación a la asamblea del sínodo, pero no es el tema principal ni el protagonista de la noticia.


## 14. Explicación sobre el significado de la IA

In [ ]:
class IaSignificadoConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No explica significado, 2=Sí explica significado")
    # Campo nuevo
    explicacion: str = Field(..., description="Justificación: ¿Hay definiciones técnicas o es solo mención?")

def clasificar_var_significado_ia(titulo: str, texto_cuerpo: str) -> IaSignificadoConExplicacion:
    """
    Detecta si el artículo explica o define QUÉ ES la IA o CÓMO FUNCIONA.
    Devuelve un objeto con .codigo (1/2) y .explicacion (str).
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()

    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    # Si no hay palabras "inteligencia" o "algoritmo", difícilmente explicará qué son.
    keywords_tecnicas = ["inteligencia", "ia ", "ai ", "algoritmo", "red neuronal", "modelo de lenguaje"]
    
    if not any(k in texto_completo for k in keywords_tecnicas):
        return IaSignificadoConExplicacion(
            codigo=1,
            explicacion="El texto no contiene términos técnicos básicos para ofrecer una definición de IA."
        )

    # --- 2. PROMPT EN FORMATO JSON ---
    # Recortamos el texto (buscamos definiciones, suelen estar al principio)
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza el siguiente texto periodístico con enfoque pedagógico:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea: Determinar si el artículo contiene una EXPLICACIÓN o DEFINICIÓN sobre qué es la Inteligencia Artificial (IA) o cómo funciona técnicamente.

    Criterios de clasificación:
    
    1 = No (Mera mención o uso):
        - El artículo habla de herramientas (ChatGPT, Bard) o noticias de empresas sin explicar qué son.
        - Habla de "la IA" como un sujeto abstracto ("la IA cambiará el mundo") sin definirla.
        - Ej: "Google lanzó su nueva IA ayer". Aquí NO se aprende qué es la tecnología.

    2 = Sí (Didáctico / Definitorio):
        - El texto tiene intención educativa.
        - Contiene frases tipo: "La IA generativa funciona prediciendo el siguiente token...", "Los LLM son modelos entrenados con...".
        - Explica la diferencia técnica entre tipos de IA.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Breve frase justificando si hay definiciones técnicas o solo menciones)"
    }}
    """

    # --- 3. LLAMADA AL MODELO ---
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. EXTRACCIÓN Y VALIDACIÓN ---
    try:
        # Buscamos el bloque JSON por si el modelo añade texto extra
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            return IaSignificadoConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato JSON válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        # Validación con Pydantic
        return IaSignificadoConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        return IaSignificadoConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
significado_ia = clasificar_var_significado_ia(articulo.title, articulo.text)

print(f"Código: {significado_ia.codigo}")
print(f"Razón: {significado_ia.explicacion}")

Código: 1
Razón: El artículo menciona el uso de la IA a través de referencias al Papa Francisco y sus discursos, pero no ofrece ninguna definición técnica o explicación de cómo funciona la IA.


## 15. Se menciona la IA en algún momento

In [ ]:
class MencionIaConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No menciona IA, 2=Sí menciona IA")
    # Explicación generada automáticamente por Python
    explicacion: str = Field(..., description="Justificación exacta (qué palabra o sigla se encontró)")


def clasificar_var_menciona_ia(titulo: str, texto_cuerpo: str) -> MencionIaConExplicacion:
    """
    Detecta si se menciona la IA y lista TODAS las palabras clave encontradas.
    Usa Regex y palabras clave (No requiere Ollama).
    """
    
    # Unificamos texto
    texto_completo = f"{titulo} \n {texto_cuerpo}"
    texto_lower = texto_completo.lower()
    
    # Creamos un conjunto (set) para evitar palabras repetidas
    palabras_encontradas = set()
    
    # --- 1. BÚSQUEDA DE CONCEPTOS CLAROS ---
    palabras_clave = [
        "inteligencia artificial", "artificial intelligence",
        "machine learning", "aprendizaje automático", "deep learning",
        "redes neuronales", "chatgpt", "generative ai", "ia generativa",
        "openai", "midjourney", "dall-e", "bard", "gemini", "copilot",
        "large language model", " llm ", "algoritmos generativos",
        "sam altman", "sora", "claude 3", "llama 3", "mistral",
        "vision pro", "neuralink"
    ]
    
    # Iteramos sobre todas las palabras y si están, las añadimos al set
    for frase in palabras_clave:
        if frase in texto_lower:
            palabras_encontradas.add(frase.strip()) # strip para quitar espacios de " llm "

    # --- 2. BÚSQUEDA DE SIGLAS "IA" o "AI" (Case Sensitive) ---
    # Usamos re.findall para encontrar TODAS las ocurrencias, no solo la primera
    patron_siglas = r'\b(IA|AI|I\.A\.|A\.I\.)\b'
    coincidencias_siglas = re.findall(patron_siglas, texto_completo)
    
    # Añadimos las siglas encontradas al conjunto
    for sigla in coincidencias_siglas:
        palabras_encontradas.add(sigla)

    # --- 3. CONSTRUCCIÓN DE RESPUESTA ---
    
    # Si el conjunto tiene elementos, es un SÍ (2)
    if palabras_encontradas:
        # Convertimos el set a una lista ordenada y la unimos con comas
        lista_final = ", ".join(sorted(palabras_encontradas))
        return MencionIaConExplicacion(
            codigo=2,
            explicacion=lista_final
        )

    # Si el conjunto está vacío, es un NO (1)
    return MencionIaConExplicacion(
        codigo=1,
        explicacion="No se encontraron términos, siglas ni conceptos relacionados con la Inteligencia Artificial en el texto."
    )

In [ ]:
# -- Uso --
menciona_ia = clasificar_var_menciona_ia(articulo.title, articulo.text)

print(f"Código: {menciona_ia.codigo}")
print(f"Razón: {menciona_ia.explicacion}")

Código: 2
Razón: inteligencia artificial


## 16. Referencias en la noticia a políticas en materia de género e igualdad

In [ ]:
class ReferenciaPoliticasGeneroConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No referencia políticas, 2=Sí referencia políticas de género")
    # Campo nuevo para el razonamiento
    explicacion: str = Field(..., description="Justificación de la decisión")


def clasificar_var_referencia_politicas_genero(titulo: str, texto_cuerpo: str) -> ReferenciaPoliticasGeneroConExplicacion:
    """
    Detecta si la noticia hace referencia a POLÍTICAS, LEYES o DEBATES sobre 
    género, igualdad, feminismo o violencia machista.
    Devuelve objeto con .codigo y .explicacion.
    """
    
    # Unificamos y limpiamos texto para el filtro
    texto_completo = (titulo + " " + texto_cuerpo).lower()
    
    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    raices_clave = [
        "igualdad", "género", "genero", "femin", "mujer", "machis", 
        "brecha", "paridad", "sexis", "patriarca", "trans ", "lgtbi",
        "conciliación", "techo de cristal", "violencia vicaria", "víctima",
        "ley", "ministerio", "protesta", "derechos" # Añadido contexto político/legal
    ]
    
    # Verificamos si hay al menos una palabra de género Y contexto político/social
    # (Simplificado: si no hay ninguna raíz clave, descartamos).
    if not any(raiz in texto_completo for raiz in raices_clave):
        return ReferenciaPoliticasGeneroConExplicacion(
            codigo=1,
            explicacion="El texto no contiene términos clave relacionados con género, igualdad o políticas sociales."
        )

    # --- 2. PROMPT ESPECÍFICO (Modo JSON) ---
    # Recortamos el texto para no saturar el contexto
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza la siguiente noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea: Determinar si el texto hace referencia a POLÍTICAS DE GÉNERO, LEYES DE IGUALDAD o DEBATES SOBRE DERECHOS DE LA MUJER.

    Criterios de clasificación:
    1 = No:
        - La mujer es mencionada solo como protagonista de un hecho (ej: "La alcaldesa inauguró la feria").
        - Sucesos o crímenes sin contexto social/legal.
    
    2 = Sí:
        - Menciona leyes, cuotas, paridad o medidas gubernamentales sobre igualdad.
        - Habla de violencia machista/género como problema estructural o legal.
        - Trata sobre feminismo, 8M, brecha salarial o discriminación laboral.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Frase breve justificando si se trata de política/derechos o es solo una mención circunstancial)"
    }}
    """

    # --- 3. Llamada al modelo ---
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. Procesamiento JSON ---
    try:
        # Buscamos el JSON dentro de la respuesta (por si el modelo añade texto extra)
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            # Fallback si no hay JSON
            return ReferenciaPoliticasGeneroConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        # Validación final con Pydantic
        return ReferenciaPoliticasGeneroConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        return ReferenciaPoliticasGeneroConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
referencia_politicas_genero = clasificar_var_referencia_politicas_genero(articulo.title, articulo.text)

print(f"Código: {referencia_politicas_genero.codigo}")
print(f"Razón: {referencia_politicas_genero.explicacion}")

Código: 2
Razón: El texto aborda la violencia contra las mujeres como un problema estructural, refiriéndose a la necesidad de 'espacio' para las mujeres en la Iglesia y a la importancia de 'cuidarlas, valorarlas' y 'ser generativa a través de una pastoral hecha de cuidado y solicitud'. Además, menciona la influencia de documentos como la encíclica Lumen Gentium, vinculada al Concilio Vaticano II, y el papel de María como modelo, directamente relacionado con la defensa de los derechos de la mujer.


## 17. Denuncia a la desigualdad de género

In [ ]:
class DenunciaDesigualdadConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No denuncia, 2=Sí denuncia desigualdad")
    # Nueva explicación
    explicacion: str = Field(..., description="Justificación de por qué se considera denuncia o no")


def clasificar_var_denuncia_desigualdad_genero(titulo: str, texto_cuerpo: str) -> DenunciaDesigualdadConExplicacion:
    """
    Detecta si la noticia DENUNCIA desigualdad y explica por qué.
    Devuelve objeto con .codigo (1/2) y .explicacion (str).
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()
    
    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    palabras_activadoras = [
        "desigualdad", "discriminaci", "brecha", "violencia", "machis", 
        "patriarca", "acos", "abus", "víctima", "feminici", "sexismo",
        "techo de cristal", "precariedad", "injusticia", "derechos de las mujeres",
        "igualdad real", "conciliación", "paridad"
    ]
    
    # Si NO hay palabras clave, retornamos 1 directamente con explicación automática
    if not any(p in texto_completo for p in palabras_activadoras):
        return DenunciaDesigualdadConExplicacion(
            codigo=1, 
            explicacion="El texto no contiene términos relacionados con género, desigualdad o violencia machista."
        )

    # --- 2. PROMPT PARA JSON ---
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza el enfoque de esta noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea es determinar si el texto DENUNCIA o VISIBILIZA un problema de desigualdad de género.

    Criterios:
    1 = No (Neutro/Sucesos): Solo narra hechos sin crítica social, o habla de mujeres exitosas sin mencionar dificultades de género.
    2 = Sí (Denuncia/Crítica): Critica el machismo, aporta datos de brechas, denuncia violencia sistémica o cubre protestas feministas.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Frase breve justificando si hay denuncia social o es meramente informativo)"
    }}
    """

    # --- 3. LLAMADA A OLLAMA ---
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. PROCESAMIENTO DE RESPUESTA ---
    try:
        # Limpieza de bloques de código markdown si los hay
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            # Fallback si el modelo no devuelve JSON
            return DenunciaDesigualdadConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        # Validación Pydantic
        return DenunciaDesigualdadConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        # Si algo falla en el parseo, devolvemos un objeto seguro
        return DenunciaDesigualdadConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )
    

In [ ]:
# -- Uso --
denuncia_desigualdad_genero = clasificar_var_denuncia_desigualdad_genero(articulo.title, articulo.text)

print(f"Código: {denuncia_desigualdad_genero.codigo}")
print(f"Razón: {denuncia_desigualdad_genero.explicacion}")

Código: 2
Razón: El texto denuncia la violencia contra las mujeres y, específicamente, la necesidad de que la Iglesia dé más 'espacio' a las mujeres, invocando la figura de María como modelo.  Se refiere a la 'violación de Dios' causada por la violencia contra mujeres y al imperativo de que la Iglesia se 'redescubra su rostro femenino', indicando una crítica a las estructuras y prácticas que perpetúan la desigualdad de género.


## 18. Presencia de mujeres racializadas en la noticia

In [ ]:
class MujeresRacializadasConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No aparecen, 2=Sí aparecen mujeres racializadas")
    # Justificación
    explicacion: str = Field(..., description="Detalle sobre quiénes son las mujeres detectadas y su contexto étnico")

def clasificar_var_mujeres_racializadas_noticias(titulo: str, texto_cuerpo: str) -> MujeresRacializadasConExplicacion:
    """
    Detecta la presencia o mención de mujeres racializadas (no blancas/caucásicas) en la noticia.
    Devuelve objeto con .codigo (1/2) y .explicacion (str).
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()

    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    # Buscamos términos que sugieran diversidad étnica, racial o contextos migratorios.
    # Si no aparece NADA de esto, asumimos que se habla de mujeres blancas o el tema no es racial.
    
    terminos_clave = [
        "racializada", "negra", "afro", "etnia", "raza", "indígena", 
        "gitana", "romaní", "latina", "hispana", "asiática", "árabe", 
        "musulmana", "morocc", "marroquí", "subsahariana", "migrante", 
        "refugiada", "islam", "velo", "hijab", "mestiza", "mulata",
        "origen", "nacionalidad", "extranjera", "diversidad"
    ]
    
    # Nota: Este filtro es laxo para no descartar falsos negativos, 
    # pero ayuda a limpiar noticias de política nacional estándar (ej: Ayuso, Montero).
    if not any(t in texto_completo for t in terminos_clave):
        return MujeresRacializadasConExplicacion(
            codigo=1,
            explicacion="El texto no contiene marcadores explícitos de diversidad étnica o racial."
        )

    # --- 2. PROMPT CON SOLICITUD DE JSON ---
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza la representación de las personas en esta noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea: Determinar si en la noticia aparecen, se mencionan o protagonizan **MUJERES RACIALIZADAS**.
    
    Definición de "Mujer Racializada" para este análisis:
    Mujeres que son percibidas socialmente como no blancas en un contexto occidental. Incluye:
    - Mujeres negras / afrodescendientes.
    - Mujeres latinas / sudamericanas.
    - Mujeres asiáticas.
    - Mujeres árabes / magrebíes / musulmanas (contexto cultural-étnico).
    - Mujeres indígenas.
    - Mujeres gitanas / romaníes.

    Criterios de clasificación:
    
    1 = No:
        - Solo aparecen mujeres blancas / caucásicas (ej: políticas europeas, actrices de Hollywood blancas).
        - No se menciona el origen étnico y por el contexto se asume hegemonía blanca.
        - Se habla de "inmigrantes" en general sin especificar mujeres.

    2 = Sí:
        - Aparece explícitamente una mujer descrita por su etnia u origen (ej: "la activista afroamericana", "la cantante colombiana").
        - Se menciona a una figura pública conocida por ser racializada (ej: Kamala Harris, Rihanna, Salma Hayek, Zendaya) aunque no se diga su raza explícitamente en el texto.
        - Noticias sobre colectivos específicos (ej: "Las mujeres afganas", "Las temporeras marroquíes").

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Indica qué mujer o colectivo racializado se ha detectado y por qué)"
    }}
    """

    # --- 3. LLAMADA AL MODELO ---
    respuesta_texto = consultar_ollama(prompt, modelo="gemma:4b")

    # --- 4. EXTRACCIÓN Y VALIDACIÓN ---
    try:
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            return MujeresRacializadasConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato JSON válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        return MujeresRacializadasConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        return MujeresRacializadasConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
mujeres_racializadas_noticias = clasificar_var_mujeres_racializadas_noticias(articulo.title, articulo.text)

print(f"Código: {mujeres_racializadas_noticias.codigo}")
print(f"Razón: {mujeres_racializadas_noticias.explicacion}")

Código: 1
Razón: El texto no contiene marcadores explícitos de diversidad étnica o racial.


## 19. Presencia de mujeres con discapacidad en la noticia

In [ ]:
class MujeresConDiscapacidadConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No aparecen, 2=Sí aparecen mujeres con discapacidad")
    # Justificación
    explicacion: str = Field(..., description="Detalle sobre quiénes son las mujeres detectadas y su contexto de discapacidad")

def clasificar_var_mujeres_con_discapacidad_noticias(titulo: str, texto_cuerpo: str) -> MujeresConDiscapacidadConExplicacion:
    """
    Detecta la presencia o mención explícita de mujeres con discapacidad o diversidad funcional.
    Devuelve objeto con .codigo (1/2) y .explicacion (str).
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()

    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    # Palabras clave que sugieren discapacidad, diversidad funcional o condiciones específicas.
    # Si no aparece ninguna, descartamos la noticia.
    
    terminos_clave = [
        "discapacidad", "diversidad funcional", "silla de ruedas", "movilidad reducida",
        "ciega", "sorda", "sordomuda", "invidente", "autis", " tea ", "asperger",
        "síndrome de down", "parálisis", "cerebral", "amputada", "prótesis",
        "salud mental", "trastorno", "bipolar", "esquizofren", "depresio", # En contextos de discapacidad psicosocial
        "dependencia", "capacitism", "paralímpic", "once", "cermi"
    ]
    
    if not any(t in texto_completo for t in terminos_clave):
        return MujeresConDiscapacidadConExplicacion(
            codigo=1,
            explicacion="El texto no contiene términos relacionados con la discapacidad o diversidad funcional."
        )

    # --- 2. PROMPT CON SOLICITUD DE JSON ---
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza la representación de las personas en esta noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea: Determinar si en la noticia aparecen, se mencionan o protagonizan **MUJERES CON DISCAPACIDAD**.

    Criterios de clasificación:
    
    1 = No:
        - Se menciona discapacidad, pero en HOMBRES (ej: "El atleta paralímpico ganó el oro").
        - Se usan términos metafóricos (ej: "La justicia es ciega", "parálisis política").
        - Son lesiones temporales (ej: "La jugadora se rompió la pierna y estará baja un mes").
        - Se habla de discapacidad en general (leyes, barreras) sin mencionar a ninguna mujer o colectivo femenino específico.

    2 = Sí:
        - Aparece una mujer (o niña) con discapacidad física, sensorial, intelectual o psicosocial.
        - Se habla de colectivos específicos (ej: "Las mujeres con discapacidad sufren más violencia").
        - Se menciona a deportistas paralímpicas, activistas con diversidad funcional, etc.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Indica quién es la mujer y cuál es su discapacidad o contexto)"
    }}
    """

    # --- 3. LLAMADA AL MODELO ---
    # Gemma 4b suele ser bueno distinguiendo género en estos contextos
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. EXTRACCIÓN Y VALIDACIÓN ---
    try:
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            return MujeresConDiscapacidadConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato JSON válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        return MujeresConDiscapacidadConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        return MujeresConDiscapacidadConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
mujeres_con_discapacidad_noticias = clasificar_var_mujeres_con_discapacidad_noticias(articulo.title, articulo.text)

print(f"Código: {mujeres_con_discapacidad_noticias.codigo}")
print(f"Razón: {mujeres_con_discapacidad_noticias.explicacion}")

Código: 1
Razón: El texto no contiene términos relacionados con la discapacidad o diversidad funcional.


## 20. Presencia de diversidad generacional en las mujeres que aparecen

In [ ]:
class MujeresGeneracionalidadConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No hay diversidad generacional, 2=Sí hay diversidad (niñas, ancianas o mezcla)")
    # Justificación
    explicacion: str = Field(..., description="Detalle de las edades o generaciones identificadas en la noticia")

def clasificar_var_mujeres_generacionalidad_noticias(titulo: str, texto_cuerpo: str) -> MujeresGeneracionalidadConExplicacion:
    """
    Detecta si en la noticia aparecen mujeres de **distintas generaciones** o de 
    **edades no hegemónicas** (niñas, adolescentes o ancianas).
    Devuelve objeto con .codigo (1/2) y .explicacion (str).
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()

    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    # Buscamos marcadores de edad extremos o relaciones intergeneracionales.
    # Si no aparecen, asumimos que son adultos estándar (lo más común en noticias).
    
    terminos_edad = [
        # Infancia / Juventud
        "niña", "adolescente", "joven", "menor", "escolar", "alumna", "estudiante", 
        "chica", "hija", "infantil", "bebé", "generación z",
        # Vejez / Tercera Edad
        "anciana", "abuela", "jubilada", "mayor", "tercera edad", "senior", 
        "vejez", "pensionista", "octogenaria", "nonagenaria", "vieja", "residencia",
        # Relacional
        "madre", "nieta", "familia", "generaciones", "intergeneracional"
    ]
    
    if not any(t in texto_completo for t in terminos_edad):
        return MujeresGeneracionalidadConExplicacion(
            codigo=1,
            explicacion="El texto no contiene términos que sugieran diversidad de edades (niñas, ancianas) o relaciones intergeneracionales."
        )

    # --- 2. PROMPT CON SOLICITUD DE JSON ---
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza la edad y las generaciones de las mujeres en esta noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea: Determinar si hay **DIVERSIDAD GENERACIONAL** en la representación femenina.

    Criterios de clasificación:
    
    1 = No (Representación Estándar):
        - Solo aparecen mujeres adultas en edad laboral típica (aprox 25-60 años). Ej: Políticas, profesionales, empresarias.
        - Se menciona "madre" solo como dato biográfico sin relevancia en la historia (ej: "es madre de dos hijos").
        - No se especifica la edad y se asume adultez.

    2 = Sí (Diversidad / Edades no hegemónicas):
        - Aparecen **Niñas o Adolescentes** con voz propia o como protagonistas.
        - Aparecen **Mujeres Mayores / Ancianas / Jubiladas** (Visibilidad de la tercera edad).
        - Hay un enfoque **Intergeneracional**: Se habla de madres e hijas, abuelas y nietas, o el impacto de un tema en distintas generaciones de mujeres.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Indica qué edades o relación generacional se ha detectado)"
    }}
    """

    # --- 3. LLAMADA AL MODELO ---
    # Usamos Gemma 4b (o tu modelo preferido)
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. EXTRACCIÓN Y VALIDACIÓN ---
    try:
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            return MujeresGeneracionalidadConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato JSON válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        return MujeresGeneracionalidadConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        return MujeresGeneracionalidadConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
mujeres_generacionalidad_noticias = clasificar_var_mujeres_generacionalidad_noticias(articulo.title, articulo.text)

print(f"Código: {mujeres_generacionalidad_noticias.codigo}")
print(f"Razón: {mujeres_generacionalidad_noticias.explicacion}")

Código: 1
Razón: La noticia se centra en el Papa Francisco y en su visión sobre el papel de la mujer en la Iglesia. Si bien se menciona el papel de María y de las madres, no hay ninguna mención explícita a niñas, adolescentes o mujeres mayores. La representación femenina se limita a la figura de la Virgen y a la referencia general a las madres sin detalles sobre sus edades o generaciones.


## 21. Tiene fotografías

In [29]:
class FotografiasValidadas(BaseModel):
    # Variable 1: ¿Tiene fotos? (1=No, 2=Sí)
    tiene_fotos_codigo: int = Field(..., ge=1, le=2, description="1=No, 2=Sí")
    
    # Variable 2: Número exacto de fotos
    cantidad: int = Field(..., ge=0, description="Número total de fotografías detectadas")

def clasificar_var_fotografias(articulo: Article) -> FotografiasValidadas:
    """
    Analiza las imágenes REALES (editoriales) de la noticia.
    Combina la imagen principal (top_image) + imágenes insertadas en el texto,
    filtrando iconos, logotipos y publicidad.
    """
    
    # Usamos un set para evitar duplicados de URL
    imagenes_reales = set()
    
    # 1. IMAGEN DE PORTADA (TOP IMAGE)
    # Es la más importante. Si existe, la añadimos.
    if articulo.top_image:
        imagenes_reales.add(articulo.top_image)

    # 2. IMÁGENES DEL CUERPO (CLEAN_TOP_NODE)
    # En lugar de buscar en todo el HTML, buscamos solo en el nodo que 
    # newspaper ha detectado como "cuerpo de la noticia".
    nodo_texto = articulo.clean_top_node 
    
    if nodo_texto is not None:
        # Buscamos todas las etiquetas <img> dentro del texto limpio
        imgs_en_texto = nodo_texto.xpath('.//img')
        
        for img in imgs_en_texto:
            src = img.get('src')
            if not src:
                continue
                
            src_lower = src.lower()
            
            # --- FILTROS ANTI-BASURA ---
            
            # A. Descartar formatos que suelen ser elementos de interfaz
            if src_lower.endswith('.svg') or src_lower.endswith('.gif') or src_lower.endswith('.ico'):
                continue
                
            # B. Descartar palabras clave de elementos web (no noticias)
            palabras_prohibidas = [
                'logo', 'icon', 'avatar', 'profile', 'pixel', 'spacer', 
                'doubleclick', 'adserver', 'banner', 'button', 'social',
                'facebook', 'twitter', 'whatsapp', 'share'
            ]
            
            if any(palabra in src_lower for palabra in palabras_prohibidas):
                continue

            # C. Descartar por tamaño (si el atributo existe en el HTML)
            # Muchos iconos son 1x1, 16x16, etc.
            width = img.get('width')
            height = img.get('height')
            if width and width.isdigit() and int(width) < 50:
                continue
            if height and height.isdigit() and int(height) < 50:
                continue

            # Si pasa los filtros, es una foto editorial válida
            imagenes_reales.add(src)

    # 3. CONTEO Y CÓDIGO
    cantidad_final = len(imagenes_reales)
    
    # Si hay al menos 1 foto, el código es 2 (Sí). Si es 0, es 1 (No).
    codigo_final = 2 if cantidad_final > 0 else 1

    # 4. RETORNO SEGURO
    try:
        return FotografiasValidadas(
            tiene_fotos_codigo=codigo_final,
            cantidad=cantidad_final
        )
    except ValidationError:
        # Fallback por seguridad
        return FotografiasValidadas(tiene_fotos_codigo=1, cantidad=0)

In [30]:
# --- Uso ---
tiene_fotografías = clasificar_var_fotografias(articulo).tiene_fotos_codigo
print(f"tiene_fotografías: {tiene_fotografías}")

tiene_fotografías: 2


## 22. Número de fotografías

In [31]:
# --- Uso ---
numero_fotografias = clasificar_var_fotografias(articulo).cantidad
print(f"numero_fotografias: {numero_fotografias}")

numero_fotografias: 3


## 23. Tiene Fuentes

## 24. Número de Fuentes

## 20. Utiliza Fuentes

In [41]:
# Detectar y extraer citas
quote_info = detect_and_extract_quotes(articulo_text)
print(f"(Pred) Cita en el titular: {quote_info}")
print(f"(Real) Cita en el titular: {articulo['Cita_en_titular']}")

print("Citas directas:", quote_info['explicit_quotes'])
print("Citas indirectas:", quote_info['indirect_quotes'])

# Generar HTML con citas y fuentes resaltadas
html_content = f"""
<style>
    .explicit-quote {{ color: red; font-weight: bold; }}
    .indirect-quote {{ color: blue; font-style: italic; }}
    .source {{ color: green; font-weight: bold; }}
    .verb {{ color: purple; }}
</style>
<p>{highlight_quotes_html(articulo_text)}</p>
"""

# Mostrar HTML en Jupyter Notebook
display(HTML(html_content))

(Pred) Cita en el titular: {'has_quote': 1, 'explicit_quotes': [{'source': 'Desconocido', 'quote': '"La felicidad, de la que se habla mucho, es una actitud"'}], 'indirect_quotes': [], 'sources': ['Agatha', 'Naoko Takeuchi', 'Moria Casán', 'Nerea', 'Carlos Latre', 'Protección Civil', 'Nerea Luis', 'Nacho', 'Fangoria', 'Fundación Inspiring Girls', 'Agatha Ruiz de la Prada', 'Sole Giménez', 'Alaska']}
(Real) Cita en el titular: 1
Citas directas: [{'source': 'Desconocido', 'quote': '"La felicidad, de la que se habla mucho, es una actitud"'}]
Citas indirectas: []


## 21. Escribe el número de declaraciones

In [42]:
# Contar declaraciones
num_explicit_quotes = len(quote_info['explicit_quotes'])
num_implicit_quotes = len(quote_info['indirect_quotes'])
total_quotes = num_explicit_quotes + num_implicit_quotes
print("(Pred) Total declaraciones:", total_quotes)
print("(Real) Total declaraciones:", articulo['ndeclaracion'])

(Pred) Total declaraciones: 1
(Real) Total declaraciones: 11


## 22. Nombre

## 23. Género persona declara

## 24. Tipo de fuente

## 25. Biografía